In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
np.random.seed(0)

In [ ]:
data=pd.read_excel('/content/drive/MyDrive/NNFL data/Data_A2/data55.xlsx')

In [ ]:
data=data.sample(frac=1,random_state=50)
data.head()

In [ ]:
dataset=np.array(data)#converting data

In [ ]:
dataset

In [ ]:
#splitting data into input and output
X=dataset[:,:-1]
y=dataset[:,-1]

#normalising dataset
m=X.shape[0]
xmin=np.min(X,axis=0)
xmax=np.max(X,axis=0)
X=(X-xmin)/(xmax-xmin)#performing normalization on input features 
pp=np.ones([m,1])
X=np.append(pp,X,axis=1)

In [ ]:
#splitting dataset using hold out cross validation of 70 10 and 20
X_train,X_valid,X_test=np.split(X,[int(.7*len(data)),int(.8*len(data))]) 
y_train,y_valid,y_test=np.split(y,[int(.7*len(data)),int(.8*len(data))])

In [ ]:
def sigmoid(a):
  fa=(1.0/(1+np.exp(-a)))
  return fa

In [ ]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

In [ ]:
def kernel_Matrix(X, Y):
    k = (1 + np.matmul(X, Y.transpose()))**7
    return k

In [ ]:
def rbf_kernel(x, y, sigma=2.0):
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

In [ ]:
def kernel_Perceptron(X, Y, iterations,kernel_name):
    n = len(X)
    lossList = []
    itrList = []
    accuracyList = []
    alphaList = []

    alpha = np.zeros((n,))
    if(kernel_name=="kernel_Matrix"):
      kernel_Mat= kernel_Matrix(X, X) 
    elif(kernel_name=="linear_kernel"):
      kernel_Mat=linear_kernel(X,Y)
    else:
      kernel_Mat=rbf_kernel(X,Y,2.0)  

    itr = 0

    while (itr < iterations):
        loss=0
        for i in range(n):
            u = np.sign(np.matmul(kernel_Mat[i][:], alpha * Y))
            if (u * Y[i] <= 0):
                alpha[i] = alpha[i] + 1

        for i in range(n):
            u = np.sign(np.matmul(kernel_Mat[i][:], alpha * Y))
            if (u * Y[i] <= 0):

                loss = loss + 1
        loss = loss * 1.0/ n
        accuracy = (1-loss)*100
        itrList.append(itr+1)
        lossList.append(loss)
        accuracyList.append(accuracy)
        alphaList.append(alpha * 1)

        itr = itr + 1
    return alphaList, lossList, itrList, accuracyList, kernel_Mat

In [ ]:
def kernel_Perceptron_valid(xtrain, ytrain, xvalid, yvalid, weightList, iterations):
    n=len(xvalid)
    itr=0
    lossList=[]
    itrList=[]
    accuracyList=[]
    kernel_Mat= kernel_Matrix(xvalid, xtrain)

    for alpha in weightList:
        loss=0
        for i in range(0,n):
            u = np.sign(np.matmul(kernel_Mat[i][:], alpha * ytrain))
            if (yvalid[i] * u <=0):
                loss = loss+1

        loss = loss * 1.0 / n
        accuracy = (1-loss)*100
        lossList.append(loss)
        itrList.append(itr+1)
        accuracyList.append(accuracy)
        itr = itr + 1
    return lossList, itrList, accuracyList

In [ ]:
def kernel_Percep(xtrain, ytrain,kernel_name):

    maxAccuracyList = []   
    if(kernel_name=="kernel_Matrix"):
      weightList, lossList, itrList, accuracyList, kernel_mat = kernel_Perceptron(xtrain, ytrain, 50, "kernel_Matrix")
    #Validation
    lossList, itrList, accuracyList = kernel_Perceptron_valid(xtrain, ytrain, X_valid, y_valid, weightList, 50)
    maxAccur = max(accuracyList)
    maxAccuracyList.append(maxAccur)

    return maxAccuracyList

In [ ]:
maxAccuracyList = kernel_Percep(X_train, y_train)
best_alpha = max(maxAccuracyList) 
print(best_alpha)

In [ ]:
best_alpha, lossList, itrList, accuracyList, kernel_mat = kernel_Perceptron(X_train, y_train, 50,"kernel_Matrix")

In [ ]:
kernel_Mat = kernel_Matrix(X_test, X_train)
alpha_test = best_alpha[5]

def testvalues_y(X, W, k,ytest):
    n = len(X_test)
    predicted_Y= []
    for i in range(n):
        u = np.sign(np.matmul(kernel_Mat[i][:], alpha_test * y_train))
        if u < 0:
            u=0
        predicted_Y.append(u)
        print("Predicted: {0}, Actual: {1}".format(int(u),int(y_test[i])))

    return predicted_Y

#get prediction values for test file
y_pred_test= testvalues_y(X_test, alpha_test, kernel_Mat,y_test)

In [ ]:
def sas(ypred, ytest):
    tp = tn = fp = fn = 0 #initialising true positive, true negative, false positive and false negative
    m = ytest.shape[0]
    for i in range(m):
        if ypred[i] == 1:
            if ytest[i] == 1:
                tp+=1
            else:
                fp+=1
        elif ypred[i] == -1:
            if ytest[i] == 0:
                tn+=1
            else:
                fn+=1
    se = tp/(tp+fn)
    sp = tn/(tn+fp)
    ac = (tn+tp)/m
    print("Sensitivity: {0}, Specificity: {1} and Accuracy: {2}".format(se,sp,ac))

sas(y_pred_test,y_test)